In [1]:
from chat_with_assistant import GDPR_AI_Assistant
from rag_qa import *
from access_token import *
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFaceEndpoint
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
import time 
import random 
from tqdm import tqdm 

c:\Users\faceru\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
rag_assistant = GDPR_AI_Assistant()
rag_assistant.create_session()
system_prompt = """Using the information contained in the context, give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question. 
Always mention the relevant GDPR articles.
"""

temperature=0.1
max_length=100

rag_assistant.set_system_parameters(prompt=system_prompt, temperature=temperature, max_length = max_length)

system_prompt_general = (f"""{system_prompt}"""
                    "\n\n"
                    "{context}")

prompt = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt_general),
                ("human", "{input}"),
            ]
            )
    
general_llm = HuggingFaceEndpoint(
        repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
        task="text-generation",
        do_sample=True,
        temperature=temperature,
        repetition_penalty=1.1,
        return_full_text=False,
        max_length=max_length,
        huggingfacehub_api_token=my_huggingface_token
        )
general_llm_assistant = prompt | general_llm 

Data Prepared


WARNING! max_length is not default parameter.
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.
WARNING! max_length is not default parameter.
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


Vector Store Created
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\faceru\.cache\huggingface\token
Login successful
Pipeline Built
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\faceru\.cache\huggingface\token
Login successful


### EXIN Privacy & Data Protection Foundation

Source: https://dam.exin.com/api/&request=asset.permadownload&id=384&type=this&token=7ceee7e9b29ba9d2a28cc46fc3298e7d

In [3]:
rag_llm_answers = []
general_llm_answers = []
for q in tqdm(question_list):
    q_ready = 'Please help me answer the following question:\n'+q
    rag_llm_answers.append(rag_assistant.chat_with_llm(q_ready, True))
    time.sleep(1+random.random()*2)
    general_llm_answers.append(general_llm_assistant.invoke({'context':'','input':q_ready}))
    time.sleep(1+random.random()*2)

100%|██████████| 40/40 [09:27<00:00, 14.18s/it]


In [77]:
rag_clean_answers = [i[i.upper().find('ANSWER')+7:i.upper().find('ANSWER')+9].strip() for i in rag_llm_answers]
general_clean_answers = [i[i.upper().find('ANSWER')+7:i.upper().find('ANSWER')+9].strip() for i in general_llm_answers]

eval_df = pd.DataFrame({'Correct':question_answers, 'RAG':rag_clean_answers, 'General':general_clean_answers})
#Fixing answers that were not extracted
eval_df.loc[1,'General']='D'
eval_df.loc[5,'General']='B'
eval_df.loc[6,'General']='A'
eval_df.loc[7,'General']='B'
eval_df.loc[9,'General']='D'
eval_df.loc[10,'General']='A'
eval_df.loc[11,'General']='C'
eval_df.loc[13,'RAG']='D'
eval_df.loc[13,'General']='D'
eval_df.loc[14,'RAG']='A'
eval_df.loc[14,'General']='A'
eval_df.loc[16,'General']='A'
eval_df.loc[17,'RAG']='D'
eval_df.loc[17,'General']='E'
eval_df.loc[18,'General']='C'
eval_df.loc[20,'RAG']='B'
eval_df.loc[20,'General']='B'
eval_df.loc[21,'General']='E'
eval_df.loc[23,'General']='A'
eval_df.loc[26,'General']='A'
eval_df.loc[27,'General']='C'
eval_df.loc[30,'General']='D'
eval_df.loc[31,'General']='D'
eval_df.loc[32,'General']='B'
eval_df.loc[33,'General']='D'
eval_df.loc[34,'RAG']='C'
eval_df.loc[34,'General']='C'
eval_df.loc[35,'General']='B'
eval_df.loc[37,'General']='B'
eval_df.loc[38,'General']='D'
eval_df.loc[39,'General']='B'

In [81]:
correct_rag = 0 
correct_general = 0
for i in range(eval_df.shape[0]):
    if eval_df.loc[i, 'RAG']==eval_df.loc[i, 'Correct']:
        correct_rag+=1
    if eval_df.loc[i, 'General']==eval_df.loc[i, 'Correct']:
        correct_general+=1
print('RAG Accuracy:', correct_rag/eval_df.shape[0])
print('General Accuracy:', correct_general/eval_df.shape[0])

RAG Accuracy: 0.625
General Accuracy: 0.675
